In [1]:
import re
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import random

import unicodedata
import unicodeblock.blocks

import djvu.decode

def isArmenian(s):
    return any(unicodeblock.blocks.of(c) == 'ARMENIAN' for c in s)

def isCyrillic(s):
    return any(unicodeblock.blocks.of(c) == 'CYRILLIC' for c in s)

def fixDoubleLetter(word):
    if word in ['դդում', 'թթու', 'կկու', 'շշուկ', 'խխունջ', 'ծծակ', 'թթվածին', 'թթվասեր']:
        return word
    if len(word) > 1:
        if word[0] == word[1]:
            return word[1:]
    return word

print(djvu.decode.DDJVU_VERSION)

26


In [2]:
context = djvu.decode.Context()
document = context.new_document(djvu.decode.FileUri('data/hayeren.djvu'))
document.decoding_job.wait()

In [3]:
pages = []

for page in document.pages:
    val = page.text.sexpr.value

    line_delims_and_words = [y for x in val if type(x) == tuple for y in x if type(y) != int]

    lines = []
    current_line = []

    for entry in line_delims_and_words[1:]:
        if type(entry) == tuple:
            current_line.append(entry[-1])
        else:
            lines.append(' '.join(current_line))
            current_line = []
    lines.append(' '.join(current_line))

    pages.append(lines)
    
# get section headers from pages 4-11, ignore page numbers in page headers
section_headers = [line for 
    page in pages[4:12] for line in page[1:-1] if any(not c in '0123456789' for c in line)][3:]

section_headers
    
core_lines = [line.strip() for page in pages[16:-1] for line in page[:-2]]

# some lines of the form 'остановить (для проверки) կանգնեցնել' mix Cyrillic and Armenian!
core_lines_fix = []
for line in core_lines:
    if isArmenian(line) and isCyrillic(line):
        core_lines_fix.append(line[:line.index(')')+1])
        core_lines_fix.append(line[line.index(')')+2:])
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

# another fix, for consecutive Armenian lines. use separate loop since logic is more complex
core_lines_fix = []
skip_steps = 0

for i in range(len(core_lines)):
    if skip_steps:
        skip_steps -= 1
        continue
    
    line = core_lines[i]
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isArmenian(line) and isArmenian(next_line):
        if next_line.startswith(line):
            pass
        elif isArmenian(core_lines[i+2]): # only one case!
            core_lines_fix.append(line + ' ' + next_line + ' ' + core_lines[i+2])
            skip_steps = 2
        else:
            core_lines_fix.append(line + ' ' + next_line)
            skip_steps = 1
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

# one more fix, for consecutive Russian lines (other than headers)
core_lines_fix = []
skip_steps = 0

for i in range(len(core_lines)):
    if skip_steps:
        skip_steps -= 1
        continue
    
    line = core_lines[i]
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isCyrillic(line) and isCyrillic(next_line):
        if line in section_headers:
            core_lines_fix.append(line)
        else:
            core_lines_fix.append(line + ' ' + next_line)
            skip_steps = 1
    else:
        core_lines_fix.append(line)
core_lines = core_lines_fix

del core_lines_fix

In [4]:
# check that all section_headers do actually appear in text
for sh in section_headers:
    print('✅' if sh in core_lines else '❌', sh)

✅ ОСНОВНЫЕ ПОНЯТИЯ
✅ основные понятия | часть 1
✅ 1. Местоимения
✅ 2. Приветствия. Прощания. Извинения. Благодарность
✅ 3. Обращения
✅ 4. Числаот1до100
✅ 5. Числа от 100
✅ 6. Числа. Порядковые числительные
✅ 7. Числа. Дроби
✅ 8. Числа. Математические действия
✅ 9. Числа. Разное
✅ 10. Самые важные глаголы - 1
✅ 11. Самые важные глаголы - 2
✅ 12. Самые важные глаголы - 3
✅ 13. Самые важные глаголы - 4
✅ 14. Цвета
✅ 15. Вопросы
✅ 16. Основные предлоги
✅ 17. Вводные и служебные слова. Наречия - 1
✅ 18. Вводные и служебные слова. Наречия - 2
✅ основные понятия | часть 2
✅ 19. Дни недели
✅ 20. Часы. Время суток
✅ 21. Месяцы. Времена года
✅ 22. Время. Разное
✅ 23. Противоположности
✅ 24. Линии и формы
✅ 25. Меры измерения
✅ 26. Ёмкости
✅ 27. Материалы
✅ 28. Металлы
✅ ЧЕЛОВЕК
✅ человек | тело человека
✅ 29. Человек. Общие понятия
✅ 30. Анатомия
✅ 31. Голова
✅ 32. Тело
✅ одежда | аксессуары
✅ 33. Верхняя одежда
✅ 34. Одежда
✅ 35. Одежда. Бельё
✅ 36. Головной убор
✅ 37. Обувь
✅ 38. Ткани. Матери

In [5]:
category = ''
subcategory = ''
group = ''

dfRows = []

for i in range(len(core_lines)):
    line = core_lines[i]
    
    if isArmenian(line):
        continue # handle all processing one line ahead of definitions
    
    if i < len(core_lines) - 1:
        next_line = core_lines[i+1]
    else:
        next_line = ''
    
    if isCyrillic(line) and isCyrillic(next_line):
        if re.match(r'^\d+\. ', line):
            group = line
        if line.islower():
            subcategory = line
        if line.isupper():
            category = line
    
    if isArmenian(next_line):
        dfRows.append({
            'category': category, 'subcategory': subcategory, 'group': group,
            'russian': line, 'armenian': next_line
        })

Phrasebook = DataFrame(dfRows)

Phrasebook.armenian = Phrasebook.armenian.apply(fixDoubleLetter)

In [6]:
Phrasebook.sample(30)

,category,subcategory,group,russian,armenian
8503,РАЗНОЕ,основные глаголы | 550 слов,253. Глаголы Ж-М,любить (человека),սիրել
7877,СТРАНОВЕДЕНИЕ,страны | национальности,244. Политика. Правительство - 2,экономический спад,տնտեսական անկում
2657,ЧЕЛОВЕК | СРЕДА ОБИТАНИЯ,жилище,86. Дом. Жилище,дома (у себя),տանը
3769,ДЕЯТЕЛЬНОСТЬ ЧЕЛОВЕКА,профессии | занятия,123. Поиск работы. Увольнение,уйти на пенсию,թոշակի գնալ
6872,ПРИРОДА,фауна,213. Собака. Породы,бульдог,բուլդոգ
6058,ЛЮДИ | СОБЫТИЯ,события в жизни человека,190. Дорога. Путь,дорога,ճանապարհ
801,ОСНОВНЫЕ ПОНЯТИЯ,основные понятия | часть 2,25. Меры измерения,штука,հատ
2428,ЧЕЛОВЕК | СРЕДА ОБИТАНИЯ,город,79. Городские учреждения,собор,տաճար
7597,СТРАНОВЕДЕНИЕ,страны | национальности,237. Азия,кореянка,կորեուհի
7082,ПРИРОДА,фауна,221. Животные. Части тела,копыто,սմբակ


In [18]:
Phrasebook[Phrasebook.group.str.startswith('249.')].sample(20)

,category,subcategory,group,russian,armenian
8128,РАЗНОЕ,термины общего характера,249. Термины общего характера,усилие,ջանք
8115,РАЗНОЕ,термины общего характера,249. Термины общего характера,сравнение,համեմատություն
8135,РАЗНОЕ,термины общего характера,249. Термины общего характера,элемент,տարր
8120,РАЗНОЕ,термины общего характера,249. Термины общего характера,степень,աստիճան
8090,РАЗНОЕ,термины общего характера,249. Термины общего характера,пауза,դադար
8072,РАЗНОЕ,термины общего характера,249. Термины общего характера,вариант,տարբերակ
8099,РАЗНОЕ,термины общего характера,249. Термины общего характера,проблема,խնդիր
8108,РАЗНОЕ,термины общего характера,249. Термины общего характера,свойство,հատկություն
8130,РАЗНОЕ,термины общего характера,249. Термины общего характера,фон,ֆոն
8083,РАЗНОЕ,термины общего характера,249. Термины общего характера,начало,սկիզբ


In [8]:
[word for word in Phrasebook.armenian if 
                             word[0] == word[1] and not word[0] == '.']

['թթվասեր',
 'դդում',
 'ծծակ',
 'թթվածին',
 'շշուկ',
 'կկու',
 'խխունջ',
 'դդում',
 'թթու']

In [16]:
Phrasebook.group.value_counts()[:20]

253. Глаголы Ж-М                              97
251. Определения О-Я                          96
250. Определения А-Н                          93
252. Глаголы А-Е                              90
234. Западная Европа                          90
248. Вера. Христианство. Ислам                88
256. Глаголы Р-Т                              88
255. Глаголы П                                82
17. Вводные и служебные слова. Наречия - 1    79
254. Глаголы Н-О                              78
168. Инструменты                              78
237. Азия                                     76
136. Футбол                                   76
242. Города                                   73
44. Продукты                                  73
117. Отрасли и виды бизнеса                   71
70. Обиды. Ссора. Негативные эмоции           70
23. Противоположности                         68
171. Корабль                                  68
249. Термины общего характера                 67
Name: group, dtype: 

In [20]:
Phrasebook.armenian.value_counts()[:20]

կրակել                 9
խաղալ                  7
հետեւել                7
վիրավորել              7
հաշիվ                  7
միացնել                6
ներողություն խնդրել    6
հեռացնել               6
վարձել                 6
լսել                   6
փորձել                 6
հավաքել                6
զանգ                   6
ցույց տալ              6
բացել                  6
ուղղել                 6
տաք                    6
քննարկել               6
հանգիստ                6
ներել                  6
Name: armenian, dtype: int64

In [24]:
Phrasebook[Phrasebook.armenian=='հեռացնել']

,category,subcategory,group,russian,armenian
2274,ЧЕЛОВЕК,медицина,74. Симптомы болезней. Лечение - 3,удалять (зуб),հեռացնել
3764,ДЕЯТЕЛЬНОСТЬ ЧЕЛОВЕКА,профессии | занятия,123. Поиск работы. Увольнение,уволить,հեռացնել
5055,ТЕХНИКА | ТРАНСПОРТ,техника,165. Компьютер,удалить,հեռացնել
5096,ТЕХНИКА | ТРАНСПОРТ,техника,166. Интернет. E-mail,удалить,հեռացնել
8769,РАЗНОЕ,основные глаголы | 550 слов,257. Глаголы У-Я,удалять (пятно и т.п .),հեռացնել
8796,РАЗНОЕ,основные глаголы | 550 слов,257. Глаголы У-Я,устранять (препятствие),հեռացնել
